In [ ]:
import pandas as pd
import tables
from analysisfunctions import *
from tenthRound import getTenthRoundKey


In [ ]:
f = tables.open_file("aes.hdf5", 'r')

In [ ]:
print(f.root)
faultgroup = f.root
for n in faultgroup._f_iter_nodes('Group'):
    print(n)
[success, fail] = filter_endstatus_status(f.root.fault)
print(len(success))
print(len(fail))

In [ ]:
reggroups = filter_experiment_type(f.root.fault, 2, success)
print(len(reggroups))
print(reggroups)

In [ ]:
def findout_aes_faulttype(faultgroup, goldengroup):
    noerror = []
    oneerror = []
    fourerror = []
    allerror = []
    notmatched = []
    garray = goldengroup.memdumps.location_20000040_16_1[:][0]
    for node in faultgroup._f_iter_nodes('Group'):
        array_node = node.memdumps.location_20000040_16_1
        array = array_node[:][0]
        diff = 0
        for i in range(0,16):
            if array[i] != garray[i]:
                diff = diff + 1
        if diff == 0:
            noerror.append(node._v_name)
        elif diff == 1:
            oneerror.append(node._v_name)
        elif diff == 4:
            fourerror.append(node._v_name)
        elif diff == 16:
            allerror.append(node._v_name)
        else:
            notmatched.append(node._v_name)
        #print(array)
        #print(garray)
    return [noerror, oneerror, fourerror, allerror, notmatched]

def group_exp_cyphers(groupnamelist):
    retlist = []
    for expname in groupnamelist:
        found = 0
        node = f.root.fault._f_get_child(expname)
        aes_cypher = node.memdumps.location_20000040_16_1[:][0]
        for exp in retlist:
            diff = 0
            for i in range(0, 16):
                if exp['cypher'][i] != aes_cypher[i]:
                    diff = diff + 1
                if (diff == 0) and (i == 15):
                    found = found + 1
                    exp['expgroup'].append(expname)
                    break
            if found != 0:
                break
        if found == 0:
            nodedict = {}
            tmp = []
            tmp.append(expname)
            nodedict['expgroup'] = tmp
            nodedict['cypher'] = aes_cypher
            retlist.append(nodedict)
    return retlist

In [ ]:
# Sort output in different amounts of errors in the status
[noerror, oneerror, fourerror, allerror, notmatched] = findout_aes_faulttype(f.root.fault, f.root.Goldenrun)
print("noerror: {}".format(len(noerror)))
print("oneerror: {}".format(len(oneerror)))
print("fourerror: {}".format(len(fourerror)))
print("allerror: {}".format(len(allerror)))
print("notmatched: {}".format(len(notmatched)))

In [ ]:
# Try to find all round skip attacks
instruction_faults = filter_experiment_type(f.root.fault, 1, success)
instruction_faults = filter_faultaddress_exp(f.root.fault, 134220088, 134220098, instruction_faults)
print(len(instruction_faults))
print(instruction_faults)

In [ ]:
# Combine the different outputs of the round skip attack into dictionaries

grouped_roundskips = group_exp_cyphers(instruction_faults)
print(len(grouped_roundskips))
print(grouped_roundskips)

In [ ]:
# Reverse the tenth round to get roundkey
golden_aes_cypher = f.root.Goldenrun.memdumps.location_20000040_16_1[:][0]

for groupdic in grouped_roundskips:
    groupdic['guess'] = getTenthRoundKey(groupdic['cypher'], golden_aes_cypher)

In [ ]:
# Check if a correct guess is inside the dataset for round skip attack
tenthroundkey = [208, 20, 249, 168, 201, 238, 37, 137, 225, 63, 12, 200, 182, 99, 12, 166]
correct_guess = []
for groupdic in grouped_roundskips:
    match = 0
    for i in range(0, 16):
        if groupdic['guess'][i] == tenthroundkey[i]:
            match = match + 1
    if match == 16:
        print(len(groupdic['expgroup']))
        correct_guess.append(groupdic)
print(len(correct_guess))
print(correct_guess)

In [ ]:
match_allerror = intersectlists(allerror, reggroups)
print(len(match_allerror))
deduped_match_allerror = group_exp_cyphers(match_allerror)
print(len(deduped_match_allerror))
print(deduped_match_allerror)